In [6]:
import yaml
from langchain_neo4j import Neo4jGraph
import sys

sys.path.append('C:/Users/tjker/Desktop/Research/Projects/lit_review/lit_review')
import semantic_scholar_api as ss_api
import neo4j_utils as nu
import kg_builder as kgb

def load_config(config_file="C:/Users/tjker/Desktop/Research/Projects/lit_review/configs/build_kg.yaml"):
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)
    return config

In [7]:
config = load_config()
kg = Neo4jGraph(
    url=config['database']['uri'], 
    username=config['database']['username'], 
    password=config['database']['password'],
    database=config['database']['database']
)

with open(config['file_paths']['paper_titles'], 'r') as file:
    titles = [line.strip() for line in file]

In [3]:
paper_data = ss_api.extract_paper_data(titles[:2])
author_info = nu.create_paper_nodes(kg, paper_data, return_authors=True)
if config['graph']['author']:
    for paper in author_info:
        kgb.build_author_data(kg, paper['authors'], paper['paperId'])

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Exceeded rate limit. Please try again later.


In [ ]:
if config['graph']['citation']:    
    for paper in paper_data:
        try:
            citation_data = ss_api.exponential_backoff_retry(ss_api.get_paper_references, paper['paperId'], fields=config['graph']['citation_fields'])
            for cited_paper in citation_data['data']:
                cited_paper = cited_paper['citedPaper']
                cited_paper_node = nu.search_papers_by_paperid(kg, cited_paper['paperId'])
                if len(cited_paper_node) == 0:
                    cited_paper['level'] = 2
                    paper_properties = {key: value for key, value in cited_paper.items() if key != 'authors'}
                    cited_paper_node = nu.create_paper_node(kg, paper_properties)
                nu.create_cites_rel(kg, paper['paperId'], cited_paper['paperId'])     
                if config['graph']['author']:
                    kgb.build_author_data(kg, cited_paper['authors'], cited_paper['paperId'])
        except ss_api.RateLimitExceededError:
            print("Exceeded rate limit. Please try again later.")
        except Exception as e:
            print(f"An error occurred: {e}")
        # citation_data = kgb.build_citation_data(kg, config, paper['paperId'])

[{'p': {'citationCount': 28, 'level': 1, 'publicationTypes': ['JournalArticle', 'Conference'], 'abstract': "Denoising Diffusion Models (DDMs) have emerged as a strong competitor to Generative Adversarial Networks (GANs). However, despite their widespread use in image synthesis and editing applications, their latent space is still not as well understood. Recently, a semantic latent space for DDMs, coined ‘ $h$ -space’, was shown to facilitate semantic image editing in a way reminiscent of GANs. The $h$ -space is comprised of the bottleneck activations in the DDM's denoiser across all timesteps of the diffusion process. In this paper, we explore the properties of $h$ -space and propose several novel methods for finding meaningful semantic directions within it. We start by studying unsupervised methods for revealing interpretable semantic directions in pretrained DDMs. Specifically, we show that interpretable directions emerge as the principal components in the latent space. Additionally,